In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_squared_log_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso,Ridge,LinearRegression,ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns 
from statsmodels.stats.outliers_influence import variance_inflation_factor 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("/kaggle/input/tumor-dataset/train_tumor_Preprocessed.csv")
test = pd.read_csv("/kaggle/input/tumor-dataset/test_tumor_Preprocessed.csv")
sample = pd.read_csv("/kaggle/input/sample/sample_submission.csv")

In [ ]:
train = train.drop(columns=['mass_npea'])
test = test.drop(columns=['exposed_area'])

In [ ]:
train_x = train.drop(columns=['tumor_size'])
train_y = train['tumor_size']

In [ ]:
test_x = test.drop(columns=['tumor_size'])
test_y = test['tumor_size']

In [ ]:
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = train.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(train.values, i) 
                          for i in range(len(train.columns))] 
  
print(vif_data)

In [ ]:
#Modelling

In [ ]:
def run_mod(model,train_x,train_y,test_x):
    
    ss = StandardScaler()
    train_x = ss.fit_transform(train_x)
    test_x = ss.fit_transform(test_x)
    
    
    train_y = np.abs(train_y)
    model.fit(train_x,train_y)
    pred_train = model.predict(train_x)
    pred_train = np.abs(pred_train)
    
    pred_test = model.predict(test_x)
    pred_test = pd.DataFrame(pred_test)
    pred_test = np.abs(pred_test)
    
    
    train_score = cross_val_score(model, train_x, train_y, cv = 10, scoring = "neg_root_mean_squared_error" )
    test_score = cross_val_score(model, test_x, pred_test, cv = 10, scoring = "neg_root_mean_squared_error" )
    
    
    print(model)
    print("RMSE of Train Data = ",np.sqrt(mean_squared_error(train_y, pred_train)))
    print("Cross validation Score train = ",np.mean(np.abs(train_score)))
    print("Cross validation Score test = ",np.mean(np.abs(test_score)))
    return


In [ ]:
model=[AdaBoostRegressor(),GradientBoostingRegressor(),RandomForestRegressor(),
       KNeighborsRegressor(),Lasso(),Ridge(),LinearRegression(),ElasticNet()
]


for i in model:
    run_mod(i,train_x,train_y,test_x)

In [ ]:
model=[AdaBoostRegressor(),GradientBoostingRegressor(),
       ,Lasso(),Ridge(),ElasticNet()
       ,RandomForestRegressor(),LinearRegression()
        KNeighborsRegressor()
]

poly = PolynomialFeatures(2)
train_x_2 = poly.fit_transform(train_x)
test_x_2 = poly.fit_transform(test_x)

for i in model:
    run_mod(i,train_x_2,train_y,test_x_2)

In [ ]:
poly = PolynomialFeatures(2)
train_x_2 = poly.fit_transform(train_x)
test_x_2 = poly.fit_transform(test_x)

model = RandomForestRegressor()


ss = StandardScaler()
train_x_ss = ss.fit_transform(train_x_2)
test_x_ss = ss.fit_transform(test_x_2)

model.fit(train_x_ss,train_y)
pred_test = model.predict(test_x_ss)

In [ ]:
sample['tumor_size'] = pred_test
sample = np.abs(sample)
sample.to_csv("result.csv", index = False)